In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [2]:
import os;
os.environ["PATH"] += os.pathsep + r'C:\Program Files (x86)';

In [3]:
browser = webdriver.Chrome()
browser.get('http://www.indiawaterportal.org/met_data/')

In [4]:
Select(browser.find_element_by_id('dataTypeSelect')).select_by_visible_text("Average temperature")
Select(browser.find_element_by_id('fromYearSelect')).select_by_visible_text("1970")
Select(browser.find_element_by_id('toYearSelect')).select_by_visible_text("2002")

In [5]:
state_elements = Select(browser.find_element_by_id('stateSelect'))

state_district_list = []
for i in range(1, len(state_elements.options), 1):
    state_elements.select_by_value(str(i))
    print(state_elements.options[i].text)
    time.sleep(2)
    for d in range(1, len(Select(browser.find_element_by_id('districtSelect')).options), 1):
        try:
            Select(browser.find_element_by_id('districtSelect')).select_by_value(str(d))
            print(str(i)+Select(browser.find_element_by_id('districtSelect')).options[d].text)
        except:
            print("None")

In [6]:
state_list = [o.text for o in state_elements.options][1:]

In [7]:
state_district_list = []
for s in state_list:
    state_elements.select_by_visible_text(s)
    time.sleep(2)
    wait = WebDriverWait(browser, 10)
    wait.until(EC.visibility_of_element_located((By.ID, 'districtSelect')))
    for d in (Select(browser.find_element_by_id('districtSelect')).options[1:]):
        try:
            Select(browser.find_element_by_id('districtSelect')).select_by_visible_text(d.text)
            state_district_list.append(tuple((s, d.text)))
        except:
            pass

In [8]:
for item in state_district_list:
    print("{}_{}".format(item[0],item[1]))
    print(type("{}_{}".format(item[0],item[1])))

ANDAMAN & NICOBAR ISLANDS_ANDAMANS
<class 'str'>
ANDAMAN & NICOBAR ISLANDS_NICOBARS
<class 'str'>
ANDHRA PRADESH_ADILABAD
<class 'str'>
ANDHRA PRADESH_ANANTAPUR
<class 'str'>
ANDHRA PRADESH_CHITTOOR
<class 'str'>
ANDHRA PRADESH_CUDDAPAH
<class 'str'>
ANDHRA PRADESH_EAST GODAVARI
<class 'str'>
ANDHRA PRADESH_GUNTUR
<class 'str'>
ANDHRA PRADESH_HYDERABAD
<class 'str'>
ANDHRA PRADESH_KARIMNAGAR
<class 'str'>
ANDHRA PRADESH_KHAMMAM
<class 'str'>
ANDHRA PRADESH_KRISHNA
<class 'str'>
ANDHRA PRADESH_KURNOOL
<class 'str'>
ANDHRA PRADESH_MAHBUBNAGAR
<class 'str'>
ANDHRA PRADESH_MEDAK
<class 'str'>
ANDHRA PRADESH_NALGONDA
<class 'str'>
ANDHRA PRADESH_NELLORE
<class 'str'>
ANDHRA PRADESH_NIZAMABAD
<class 'str'>
ANDHRA PRADESH_PRAKASAM
<class 'str'>
ANDHRA PRADESH_RANGAREDDI
<class 'str'>
ANDHRA PRADESH_SRIKAKULAM
<class 'str'>
ANDHRA PRADESH_VISAKHAPATNAM
<class 'str'>
ANDHRA PRADESH_VIZIANAGARAM
<class 'str'>
ANDHRA PRADESH_WARANGAL
<class 'str'>
ANDHRA PRADESH_WEST GODAVARI
<class 'str'>
ARUNAC

In [9]:
dataframe_collection = {}
for item in state_district_list:
    state_elements.select_by_visible_text(item[0])
    time.sleep(2)
    wait = WebDriverWait(browser, 10)
    wait.until(EC.visibility_of_element_located((By.ID, 'districtSelect')))
    try:
        Select(browser.find_element_by_id('districtSelect')).select_by_visible_text(item[1])
        browser.find_element_by_id('meanButton').click()
        time.sleep(2)
        wait.until(EC.presence_of_element_located((By.ID, 'dataTable')))
        soup = BeautifulSoup(browser.page_source, "lxml")
        table = soup.find_all('table')[1]
        dataframe_collection["{}_{}".format(item[0],item[1])] = pd.read_html(str(table), header=0)[0]
        time.sleep(2)
    except:
        pass

In [11]:
for key, value in dataframe_collection.items():
    value['State'] = str(key.split('_')[0])
    value['District'] = str(key.split('_')[1])

In [13]:
frames = dataframe_collection.values()
df_master = pd.concat(frames)

In [14]:
import pickle

with open('indiawaterportal_AvTemp.pickle', 'wb') as avTemp:
    pickle.dump(df_master, avTemp)

In [15]:
df_master

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,State,District
0,1970,22.921,26.633,29.767,32.816,34.797,28.958,27.815,26.451,26.897,26.315,22.351,19.830,ANDHRA PRADESH,ADILABAD
1,1971,22.537,25.624,27.932,32.597,33.119,28.795,28.164,27.656,27.616,26.185,22.135,20.115,ANDHRA PRADESH,ADILABAD
2,1972,21.710,24.791,29.907,32.413,34.635,32.953,28.154,27.037,27.796,26.624,23.617,23.022,ANDHRA PRADESH,ADILABAD
3,1973,23.854,26.573,30.241,34.747,36.358,31.566,27.241,26.591,27.132,26.026,22.559,21.726,ANDHRA PRADESH,ADILABAD
4,1974,21.686,25.705,29.847,33.213,33.405,30.765,28.673,26.892,27.977,26.196,22.305,20.325,ANDHRA PRADESH,ADILABAD
5,1975,21.122,25.175,28.723,32.930,35.335,30.426,27.329,26.995,26.649,26.776,21.718,19.637,ANDHRA PRADESH,ADILABAD
6,1976,21.224,25.219,28.906,32.146,33.759,30.849,27.135,26.845,27.017,27.120,26.105,23.251,ANDHRA PRADESH,ADILABAD
7,1977,21.865,26.478,30.434,32.160,32.347,31.161,27.953,26.294,27.223,26.910,25.595,21.349,ANDHRA PRADESH,ADILABAD
8,1978,22.257,25.013,28.475,32.004,35.060,29.301,26.314,25.657,27.416,27.085,25.713,21.926,ANDHRA PRADESH,ADILABAD
9,1979,23.947,24.968,28.825,31.988,32.324,31.971,27.692,26.895,27.449,27.781,25.852,22.318,ANDHRA PRADESH,ADILABAD
